In [1]:
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine

<h2> 1. Creating an engine to access the database

In [3]:
engine = create_engine('sqlite:///../data/raw/Albrodmar.db')

<h2>2. Opening all the tables on Pandas

In [46]:
df_business = pd.read_sql_query("SELECT * FROM business_info", engine)

In [47]:
df_personal = pd.read_sql_query("SELECT * FROM personal_info", engine)

In [48]:
df_rank = pd.read_sql_query('SELECT * FROM rank_info', engine)


<h4> All the df have a common column 'id' but df_rank has more lines thah the others df. Will 'outer' join the three base on 'id'


<h2 style="color:green;"> >>>> REGENERATE DATAFRAME <<<<

In [98]:
df_temp = pd.merge(df_business, df_personal, on = 'id', how = 'outer')
df_merged = pd.merge(df_temp, df_rank, on = 'id', how = 'outer')
df_merged.head(2)

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,5390,52.0,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,None,1.0,bEZOS,54 years old,52.0,M,United States,https://specials-images.forbesimg.com/imageser...,jEFF BEzos,1.0,52
1,1675,53.0,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None,2.0,GATEs,62 years old,53.0,None,None,https://specials-images.forbesimg.com/imageser...,BILL GAtes,2.0,53


<h2> 3. Start Data Wrangling

In [99]:
df_merged.isnull().sum()

id                     0
Unnamed: 0_x          52
Source                52
worth                 52
worthChange           52
realTimeWorth       2260
realTimePosition      52
lastName              52
age                  117
Unnamed: 0_y          52
gender                65
country               52
image                 52
name                   0
position              52
Unnamed: 0             0
dtype: int64

<h4>3.1 Drop columns: <ul><b>realTimeWorth</b> because is full of nulls and column</ul> <ul><b>lastName</b> because is useless having column <b>name</b></ul><ul><b>'Unnamed: 0_y'</b></ul>
    <ul><b>'Unnamed:0_x'</b></ul><ul><b>'Unnamed: 0'</b> (got same values and I don't know their meaning. There are not repeated values. Seem like they are ID columns only).</ul>

In [102]:
df_merged = df_merged.drop(['realTimeWorth', 'lastName', 'Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0'], axis = 1)

In [103]:
df_merged.head()

,id,Source,worth,worthChange,realTimePosition,age,gender,country,image,name,position
0,5390,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,1.0,54 years old,M,United States,https://specials-images.forbesimg.com/imageser...,jEFF BEzos,1.0
1,1675,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,2.0,62 years old,None,None,https://specials-images.forbesimg.com/imageser...,BILL GAtes,2.0
2,2361,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,3.0,87 years old,None,None,https://specials-images.forbesimg.com/imageser...,warREN BUFfett,3.0
3,2340,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,4.0,69 years old,None,None,https://specials-images.forbesimg.com/imageser...,bERNARD ARNAUlt,4.0
4,6891,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,5.0,1985,None,None,https://specials-images.forbesimg.com/imageser...,marK ZUCKERberg,5.0


<h4>3.2 Separete column 'Source' in column "Industry" and "Company"

In [104]:
df_merged[['industry','company_name']] = df_merged.Source.str.split(' ==>',expand = True)

#drop old column Source
df_merged = df_merged.drop(['Source'], axis = 1)

<h4>3.3 Capitalizing first letters only on column 'name'

In [105]:
df_merged['name'] = df_merged['name'].str.title()

In [106]:
df_merged.head(2)

,id,worth,worthChange,realTimePosition,age,gender,country,image,name,position,industry,company_name
0,5390,112.0 BUSD,0.0 millions USD,1.0,54 years old,M,United States,https://specials-images.forbesimg.com/imageser...,Jeff Bezos,1.0,Technology,Amazon
1,1675,90.0 BUSD,-0.001 millions USD,2.0,62 years old,None,None,https://specials-images.forbesimg.com/imageser...,Bill Gates,2.0,Technology,Microsoft


<h4>3.4 Cleaning the 'age' column removing ' years old'

In [107]:
df_merged['age'] = df_merged.age.str.replace(' years old','')

<h4>3.5 Cleaning the 'worth' column removing 'BUSD'

In [110]:
df_merged['worth'] = df_merged.worth.str.replace('BUSD','')

<h4>3.6 Changing name of column '<b>worth</b>'

In [111]:
df_merged.rename(columns = {'worth':'worth in Billion USD'}, inplace = True)

<h4>3.7 Convert data in column 'worth in Billion USD' to float

In [112]:
df_merged['worth in Billion USD'] = df_merged['worth in Billion USD'].apply(pd.to_numeric)

<h4>3.8 Convert data in column 'age' to float

In [113]:
df_merged['age'] = df_merged['age'].apply(pd.to_numeric)

<h4>3.9 Cleaning the 'worthChange' column removing ' millions UDS'

In [114]:
df_merged['worthChange']=df_merged.worthChange.str.replace(' millions USD','')

<h4>3.10 Changing name of column 'worthChange'

In [115]:
df_merged.rename(columns = {'worthChange':'worth change in millions USD'}, inplace = True)

In [116]:
df_merged.head(2)

,id,worth in Billion USD,worth change in millions USD,realTimePosition,age,gender,country,image,name,position,industry,company_name
0,5390,112.0,0.0,1.0,54.0,M,United States,https://specials-images.forbesimg.com/imageser...,Jeff Bezos,1.0,Technology,Amazon
1,1675,90.0,-0.001,2.0,62.0,None,None,https://specials-images.forbesimg.com/imageser...,Bill Gates,2.0,Technology,Microsoft


<h4>3.11 Changing 'ages' with a year of birth to years old 

In [117]:
df_merged['age'] = df_merged['age'].apply(lambda x: 2018 - x if x > 150 else x)

<h4>3.12 Changin 'gender' values to be only Male or Female

In [118]:
m_filter = df_merged['gender'] == 'M'
df_merged.loc[m_filter,'gender'] = df_merged.loc[m_filter,'gender'].replace('M','Male')

In [119]:
f_filter = df_merged['gender'] == 'F'
df_merged.loc[f_filter,'gender'] = df_merged.loc[f_filter,'gender'].replace('F','Female')

In [120]:
df_merged['gender'].value_counts()

Male      1467
None       549
Female     179
Name: gender, dtype: int64

In [128]:
df_merged[df_merged['country'].isnull()].count()

TypeError: isnull() takes 1 positional argument but 2 were given

<h4>3.13 Decide to drop all registers with 'country' null because there have no values in the rest of the registries

In [139]:
df = df_merged[pd.notnull(df_merged['country'])]
df.head()
df[df['country'] == 'None'].name

1             Bill Gates
2         Warren Buffett
3        Bernard Arnault
4        Mark Zuckerberg
6       Carlos Slim Helu
              ...       
2198         Sandy Weill
2199       Wen Pengcheng
2200         Wu Chung-Yi
2206            Zhuo Jun
2207         Olgun Zorlu
Name: name, Length: 1406, dtype: object